# Imports

In [ ]:
import numpy as np
import os
import sys
%pip install -U imgaug

from helpers import *
from CNNv2 import CNN

IN_COLAB = 'google.colab' in sys.modules

# Load images

In [ ]:
# Load the training set
if (IN_COLAB):
    from google.colab import drive
    drive.mount('/gdrive')
    root_dir = "/gdrive/My Drive/ML/training/"
else:
    root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files)
print("Loading " + str(n) + " images")
imgs = np.asarray([load_image(image_dir + files[i]) for i in range(n)])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " groundtruth")
gt_imgs = np.asarray([load_image(gt_dir + files[i]) for i in range(n)])

# Load model

In [ ]:
model = CNN(
    rootdir=root_dir,
    window_size=320,
    nb_epochs=30,
    adjust_metric='val_f1',
    save_metric='val_f1',
    train_batch_size=8,
    val_batch_size=8,
    random_seed=42)

# Split dataset

> This step is not necessary when preparing submission but it is to compare different models and/or parameters.

In [ ]:
TEST_PROPORTION = 0.1
X_train, X_test, y_train, y_test = model.split_data(imgs, gt_imgs, rate=TEST_PROPORTION, seed=42)
print(X_test.shape[0], 'test images;', X_train.shape[0], 'train images')

## Train model (and save it)

In [ ]:
#model.load(os.path.join(root_dir, 'best.h5'))
model.load('saved_weights.h5') # set initial_epoch to resume training
INITIAL_EPOCH = 0

In [ ]:
model.train(X_train, y_train, initial_epoch=INITIAL_EPOCH)

In [ ]:
model.save('saved_weights.h5')

# Test model

In [ ]:
results = model.test(X_test, y_test)

In [ ]:
print(results)
print(model.model.metrics_names)